In [1]:
import os
import yaml
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from ultralytics import YOLO
import json

# %%
# 1. Gerekli Kütüphanelerin Kurulumu ve Kontrolü
# Eğer Google Colab kullanıyorsanız, bu komutu çalıştırın.
# Eğer kendi yerel ortamınızda çalışıyorsanız ve GPU'nuz varsa, CUDA sürücülerinin doğru şekilde yüklü olduğundan emin olun.
# pip install ultralytics -q

try:
    import ultralytics
    print(f"ultralytics kütüphanesi yüklü: {ultralytics.__version__}")
except ImportError:
    print("ultralytics kütüphanesi yüklü değil. Lütfen 'pip install ultralytics' komutunu çalıştırın.")

# %%

ultralytics kütüphanesi yüklü: 8.3.141


In [ ]:
# 2. Veri Seti Hazırlığı - Klasör Yapısı ve data.yaml Oluşturma
import os
import yaml
# Veri setinizin ana klasörünü belirleyin
dataset_path = 'dataset'

# Görüntü ve etiket yolları
train_images_path = os.path.join(dataset_path, 'images', 'train')
val_images_path = os.path.join(dataset_path, 'images', 'val')
test_images_path = os.path.join(dataset_path, 'images', 'test')

# Gerekli klasörleri oluştur
os.makedirs(train_images_path, exist_ok=True)
os.makedirs(val_images_path, exist_ok=True)
os.makedirs(test_images_path, exist_ok=True)
os.makedirs(os.path.join(dataset_path, 'labels', 'train'), exist_ok=True)
os.makedirs(os.path.join(dataset_path, 'labels', 'val'), exist_ok=True)
os.makedirs(os.path.join(dataset_path, 'labels', 'test'), exist_ok=True)

# Sınıf isimleri (etiketlemede kullandığınız class_id'lere karşılık gelmeli)
class_names = [
    'saglam_ped',
    'renk_degisimi',
    'leke',
    'yapi_bozulmasi',
    'kesim_hatasi'
]

# data.yaml içeriği
data_yaml_content = {
    'path': os.path.abspath(dataset_path), # Ana veri seti yolu (mutlak yol)
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': len(class_names),
    'names': class_names
}

# data.yaml dosyasını oluştur
yaml_file_path = 'data.yaml'
with open(yaml_file_path, 'w') as file:
    yaml.dump(data_yaml_content, file, default_flow_style=False)

print(f"{yaml_file_path} dosyası başarıyla oluşturuldu.")
print(f"İçerik:\n{yaml.dump(data_yaml_content, default_flow_style=False)}")
print("\nLütfen görüntülerinizi 'dataset/images/' altındaki ilgili klasörlere,")
print("etiket dosyalarınızı (YOLO formatında .txt) 'dataset/labels/' altındaki ilgili klasörlere yerleştirin.")

# %%

In [5]:

from ultralytics import YOLO
# 3. YOLOv8 Modelini Yükleme ve Eğitme

# Önceden eğitilmiş YOLOv8 modelini yükle (nano versiyonu)
model = YOLO('yolov8n.pt')

# Modeli kendi veri setimizle eğit

# Modeli kendi veri setimizle eğit
results = model.train(
    data='C:\\Users\\EREN\\Desktop\\HACKATHON\\data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    patience=20,
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    device='cpu', # <-- Burayı DÜZELTTİK! GPU yerine CPU kullanacak.
    project='eye_pad_quality_control',
    name='yolov8n_custom_dataset',
    exist_ok=True
)

print("\nModel eğitimi tamamlandı!")

# %%

Ultralytics 8.3.141  Python-3.9.13 torch-2.7.0+cpu CPU (Intel Core(TM) i7-9750H 2.60GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\EREN\Desktop\HACKATHON\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_custom_dataset, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=20, perspective=0.0

train: Scanning C:\Users\EREN\Desktop\HACKATHON\dataset\labels\train... 21 images, 0 backgrounds, 0 corrupt: 100%|██████████| 21/21 [00:00<00:00, 750.04it/s]

train: New cache created: C:\Users\EREN\Desktop\HACKATHON\dataset\labels\train.cache
val: Fast image access  (ping: 0.10.0 ms, read: 1439.5564.8 MB/s, size: 340.8 KB)



c:\Users\EREN\Desktop\HACKATHON\yolo_env_39\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\EREN\Desktop\HACKATHON\dataset\labels\val... 5 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<00:00, 1667.45it/s]

val: New cache created: C:\Users\EREN\Desktop\HACKATHON\dataset\labels\val.cache
Plotting labels to eye_pad_quality_control\yolov8n_custom_dataset\labels.jpg... 


...

      50/50         0G     0.4402      1.725      1.113          5        640: 100%|██████████| 2/2 [00:05<00:00,  2.64s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.19it/s]

                   all          5          5      0.326      0.978      0.663      0.421



50 epochs completed in 0.082 hours.
Optimizer stripped from eye_pad_quality_control\yolov8n_custom_dataset\weights\last.pt, 6.3MB
Optimizer stripped from eye_pad_quality_control\yolov8n_custom_dataset\weights\best.pt, 6.3MB

Validating eye_pad_quality_control\yolov8n_custom_dataset\weights\best.pt...
Ultralytics 8.3.141  Python-3.9.13 torch-2.7.0+cpu CPU (Intel Core(TM) i7-9750H 2.60GHz)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.38it/s]


                   all          5          5      0.588        0.6      0.746      0.479
            saglam_ped          1          1      0.248          1      0.249      0.149
         renk_degisimi          1          1      0.448          1      0.497      0.249
                  leke          1          1      0.244          1      0.995      0.503
        yapi_bozulmasi          1          1          1          0      0.995      0.796
          kesim_hatasi          1          1          1          0      0.995      0.697
Speed: 1.5ms preprocess, 49.6ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to eye_pad_quality_control\yolov8n_custom_dataset

Model eğitimi tamamlandı!


In [6]:
# Model değerlendirme
metrics = model.val(data='C:\\Users\\EREN\\Desktop\\HACKATHON\\data.yaml')

print(f"\nModel Değerlendirme Sonuçları:")
print(f"Mean Average Precision (mAP@0.50-0.95): {metrics.box.map}") # mAP (mAP50-95)
print(f"Mean Average Precision (mAP@0.50): {metrics.box.map50}")   # mAP50
# mAP75 için doğrudan bir attr olmayabilir, genellikle mAP50 ve mAP'i kullanırız.
# Eğer mAP75'e ihtiyacın olursa, metrics.box'ın detaylarına bakmak gerekebilir,
# ama genellikle map ve map50 yeterli olur.
# print(f"Mean Average Precision (mAP@0.75): {metrics.box.map75}") # Eğer yoksa bu satırı kullanma

# Sınıf bazlı Precision ve Recall değerlerine de erişebiliriz
# print(f"Precision: {metrics.box.p}")
# print(f"Recall: {metrics.box.r}")

# Kayıp değerlerini görmek istersen (eğitim sırasında görünen loss'lar)


Ultralytics 8.3.141  Python-3.9.13 torch-2.7.0+cpu CPU (Intel Core(TM) i7-9750H 2.60GHz)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 1808.21085.0 MB/s, size: 340.8 KB)


val: Scanning C:\Users\EREN\Desktop\HACKATHON\dataset\labels\val.cache... 5 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5/5 [00:00<?, ?it/s]
c:\Users\EREN\Desktop\HACKATHON\yolo_env_39\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.08it/s]


                   all          5          5      0.588        0.6      0.746      0.479
            saglam_ped          1          1      0.248          1      0.249      0.149
         renk_degisimi          1          1      0.448          1      0.497      0.249
                  leke          1          1      0.244          1      0.995      0.503
        yapi_bozulmasi          1          1          1          0      0.995      0.796
          kesim_hatasi          1          1          1          0      0.995      0.697
Speed: 3.6ms preprocess, 56.3ms inference, 0.0ms loss, 4.1ms postprocess per image
Results saved to eye_pad_quality_control\yolov8n_custom_dataset

Model Değerlendirme Sonuçları:
Mean Average Precision (mAP@0.50-0.95): 0.4787705882352941
Mean Average Precision (mAP@0.50): 0.7462500000000001


In [9]:
# 5. Tahmin Yapma (Inference)
import os
from PIL import Image
import matplotlib.pyplot as plt
# Test edilecek bir görüntü yolu (kendi test setinizden birini seçin)
# Lütfen bu yolu kendi gerçek dosya adınızla değiştirin.
# Örnek: test_image_path = os.path.join(test_images_path, 'test_image_1.jpg')

# Eğer test_images_path klasöründe dosya yoksa veya örnek dosya yoksa, geçici bir dosya oluştur
test_files = [f for f in os.listdir(test_images_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
if test_files:
    test_image_path = os.path.join(test_images_path, test_files[0])
    print(f"Test için ilk bulunan görüntü kullanılıyor: {test_image_path}")
else:
    # Eğer test klasöründe resim yoksa, dummy bir resim oluştur
    dummy_image_path = os.path.join(test_images_path, 'dummy_test_image.jpg')
    Image.new('RGB', (640, 640), color='blue').save(dummy_image_path)
    test_image_path = dummy_image_path
    print(f"UYARI: 'dataset/images/test/' klasöründe görüntü bulunamadı. Yer tutucu görüntü '{dummy_image_path}' oluşturuldu.")
    print("Lütfen gerçek test görüntülerinizi 'dataset/images/test/' klasörüne yerleştirin.")

# Tahmin yap
results = model(test_image_path, conf=0.25, iou=0.45)

# Tahmin sonuçlarını görselleştir
for r in results:
    im_array = r.plot()
    im = Image.fromarray(im_array[..., ::-1])
    plt.figure(figsize=(10, 10))
    plt.imshow(im)
    plt.title(f"Tahmin Sonuçları: {os.path.basename(test_image_path)}")
    plt.axis('off')
    plt.show()

    if r.boxes:
        print(f"\nTespit Edilen Nesneler ({os.path.basename(test_image_path)}):")
        for box in r.boxes:
            conf = box.conf.item()
            cls_id = int(box.cls.item())
            class_name = class_names[cls_id]
            xyxy = box.xyxy[0].tolist()
            print(f"  Sınıf: {class_name} (ID: {cls_id})")
            print(f"  Güven: {conf:.2f}")
            print(f"  Kutu Koordinatları (x1, y1, x2, y2): [{xyxy[0]:.2f}, {xyxy[1]:.2f}, {xyxy[2]:.2f}, {xyxy[3]:.2f}]")
            print("-" * 20)
    else:
        print(f"\n{os.path.basename(test_image_path)} görüntüsünde herhangi bir nesne tespit edilemedi.")

# %%

Test için ilk bulunan görüntü kullanılıyor: dataset\images\test\harbikesik.jpeg

image 1/1 c:\Users\EREN\Desktop\HACKATHON\dataset\images\test\harbikesik.jpeg: 640x448 (no detections), 75.1ms
Speed: 2.5ms preprocess, 75.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 448)


<Figure size 1000x1000 with 1 Axes>


harbikesik.jpeg görüntüsünde herhangi bir nesne tespit edilemedi.


In [11]:
# 6. Sonuçları Raporlama
import json
def process_detections_and_report(image_path, results_obj, class_names_list):
    faulty_products_info = []
    is_image_faulty = False
    fault_details_list = []

    for r_item in results_obj:
        if r_item.boxes:
            for box_item in r_item.boxes:
                conf_val = box_item.conf.item()
                cls_id_val = int(box_item.cls.item())
                class_name_val = class_names_list[cls_id_val]

                if class_name_val != 'saglam_ped':
                    is_image_faulty = True
                    fault_details_list.append({
                        'class': class_name_val,
                        'confidence': conf_val,
                        'bbox': box_item.xyxy[0].tolist()
                    })
    
    if is_image_faulty:
        faulty_products_info.append({
            'image_path': image_path,
            'faults': fault_details_list
        })
        print(f"HATA TESPİT EDİLDİ: {os.path.basename(image_path)}")
        for fault in fault_details_list:
            print(f"  - Sınıf: {fault['class']}, Güven: {fault['confidence']:.2f}")
    else:
        print(f"SAĞLAM ÜRÜN: {os.path.basename(image_path)}")
    
    return faulty_products_info

current_results_for_report = model(test_image_path, conf=0.25, iou=0.45)
detected_faults_report = process_detections_and_report(test_image_path, current_results_for_report, class_names)

if detected_faults_report:
    print("\n--- Hatalı Ürün Raporu ---")
    for item_report in detected_faults_report:
        print(f"Görüntü: {item_report['image_path']}")
        for fault_report in item_report['faults']:
            print(f"  Hata Tipi: {fault_report['class']}, Güven: {fault_report['confidence']:.2f}")
        print("-" * 30)
else:
    print("\nBu çalıştırmada hatalı ürün tespit edilmedi.")

output_report_path = 'faulty_product_report.json'
with open(output_report_path, 'w') as f:
    json.dump(detected_faults_report, f, indent=4)
print(f"Rapor {output_report_path} adresine kaydedildi.")

# %%


image 1/1 c:\Users\EREN\Desktop\HACKATHON\dataset\images\test\harbikesik.jpeg: 640x448 (no detections), 70.8ms
Speed: 2.3ms preprocess, 70.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 448)
SAĞLAM ÜRÜN: harbikesik.jpeg

Bu çalıştırmada hatalı ürün tespit edilmedi.
Rapor faulty_product_report.json adresine kaydedildi.


In [ ]:
from ultralytics import YOLO
model_path = 'eye_pad_quality_control/yolov8n_custom_dataset/weights/best.pt'
model = YOLO(model_path)
model.export(format='onnx', imgsz=640, device='cpu', simplify=True)

Model yükleniyor: eye_pad_quality_control/yolov8n_custom_dataset/weights/best.pt

Model FP32 TFLite formatına dönüştürülüyor...
Ultralytics 8.3.141  Python-3.9.13 torch-2.7.0+cpu CPU (Intel Core(TM) i7-9750H 2.60GHz)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'eye_pad_quality_control\yolov8n_custom_dataset\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 9, 8400) (6.0 MB)
requirements: Ultralytics requirements ['tf_keras', 'sng4onnx>=1.0.1', 'onnx_graphsurgeon>=0.3.26', 'ai-edge-litert>=1.2.0', 'onnx2tf>=1.26.3'] not found, attempting AutoUpdate...
WARNING Retry 1/2 failed: Command 'pip install --no-cache-dir "tf_keras" "sng4onnx>=1.0.1" "onnx_graphsurgeon>=0.3.26" "ai-edge-litert>=1.2.0" "onnx2tf>=1.26.3" --extra-index-url https://pypi.ngc.nvidia.com' returned non-zero exit status 1.
WARNING Retry 2/2 failed: Command 'pip install --no-cache-dir "tf_keras" "sng4onnx>=1.0.1" "onnx_graphsurgeon>=0